# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]
filepath

'/home/aureliobarbosa/Dropbox/Estudos/nanodataengineering/engenano_project1/data/song_data/A/B/C/TRABCEI128F424C983.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARJIE2Y1187B994AB7,NaN,NaN,,Line Renaud,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,152.92036,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id','year', 'duration' ]].values[0].tolist()
song_data

['SOUPIRU12A6D4FA1E1',
 'Der Kleine Dompfaff',
 'ARJIE2Y1187B994AB7',
 0,
 152.92036]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

OK!

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARJIE2Y1187B994AB7,NaN,NaN,,Line Renaud,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,152.92036,0


In [10]:
artist_data = df[['artist_id', 'artist_name','artist_location', 'artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARJIE2Y1187B994AB7', 'Line Renaud', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

OK!

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files('data/log_data')

In [13]:
filepath = log_files[0]
filepath

'/home/aureliobarbosa/Dropbox/Estudos/nanodataengineering/engenano_project1/data/log_data/2018/11/2018-11-07-events.json'

In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Miami Horror,Logged In,Kate,F,88,Harrell,250.82730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Sometimes,200,1541548876796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
1,The White Stripes,Logged In,Kate,F,89,Harrell,241.86730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,My Doorbell (Album Version),200,1541549126796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
2,Juan Carmona,Logged In,Kate,F,90,Harrell,331.44118,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Panales de Algodon,200,1541549367796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
3,Alison Krauss / Union Station,Logged In,Kate,F,91,Harrell,171.04934,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Restless,200,1541549698796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
4,Bullet For My Valentine,Logged In,Kate,F,92,Harrell,235.65016,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Begging For Mercy,200,1541549869796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
filter = df['page']=='NextSong'
df_time_table = df[filter]
print(f"Number of rows in DataFrame: {len(df_time_table)}. Sample: ")
df_time_table.sample(1)

Number of rows in DataFrame: 174. Sample: 


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
101,Ol' Dirty Bastard,Logged In,Lily,F,20,Koch,217.49506,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,Brooklyn Zoo [Explicit Version],200,1541601764796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [16]:
# testing witha single point
pd.to_datetime(1541548876796, origin='unix', unit='ms')

Timestamp('2018-11-07 00:01:16.796000')

In [17]:
t = pd.to_datetime(df_time_table['ts'],  origin='unix', unit='ms')
t.sample(5)

120   2018-11-07 16:08:34.796
36    2018-11-07 05:35:31.796
13    2018-11-07 00:46:01.796
122   2018-11-07 16:19:17.796
45    2018-11-07 06:45:31.796
Name: ts, dtype: datetime64[ns]

In [18]:
time_data = (t, 
             t.dt.hour, 
             t.dt.day, 
             t.dt.isocalendar().week, 
             t.dt.month, 
             t.dt.year, 
             t.dt.weekday, 
            )
column_labels = ('timestamp', 'hour', 'day','weekofyear','month', 'year', 'weekday')
dict_time_data = dict(zip(column_labels, time_data))

In [19]:
time_df = pd.DataFrame(dict_time_data)

In [20]:
time_df.sample(n=7)

,timestamp,hour,day,weekofyear,month,year,weekday
3,2018-11-07 00:14:58.796,0,7,45,11,2018,2
26,2018-11-07 02:56:26.796,2,7,45,11,2018,2
112,2018-11-07 15:41:10.796,15,7,45,11,2018,2
196,2018-11-07 21:05:54.796,21,7,45,11,2018,2
147,2018-11-07 17:49:16.796,17,7,45,11,2018,2
90,2018-11-07 14:16:39.796,14,7,45,11,2018,2
83,2018-11-07 13:53:37.796,13,7,45,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
df.sample(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
64,None,Logged In,Lily,F,0,Koch,NaN,paid,"Chicago-Naperville-Elgin, IL-IN-WI",GET,Home,1.541048e+12,221,None,200,1541596856796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [23]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()
user_df.drop_duplicates(subset='userId',inplace=True)
user_df.dropna(inplace=True)
print(f'user_df size: {len(user_df)}')
user_df.sample(1)

user_df size: 27


,userId,firstName,lastName,gender,level
167,52,Theodore,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### 5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`



In [25]:
df.sample(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
193,Way Out West,Logged In,Lily,F,101,Koch,657.73669,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,Killa,200,1541623631796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
146,William Orbit,Logged In,Lily,F,60,Koch,317.96200,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,Barber's Adagio For Strings (Ferry Corsten Remix),200,1541612639796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
11,Fall Out Boy,Logged In,Kate,F,98,Harrell,172.01587,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Don't You Know Who I Think I Am?,200,1541551208796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
80,Cartola,Logged In,Lily,F,8,Koch,188.65587,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,Verde Que Te Quero Rosa,200,1541598601796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
132,Bullet For My Valentine,Logged In,Lily,F,47,Koch,348.39465,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,Tears Don't Fall,200,1541609825796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [27]:
df['location']

0      1541548876796
1      1541549126796
2      1541549367796
3      1541549698796
4      1541549869796
           ...      
197    1541629595796
198    1541629614796
199    1541629964796
200    1541632356796
201    1541632589796
Name: ts, Length: 202, dtype: int64

In [34]:
for index, row in df.iterrows():
    print(index, row)
#     # get songid and artistid from song and artist tables
#     cur.execute(song_select, (row.song, row.length))
#     results = cur.fetchone()
#     print(song_select, results)  #######

#     if results:
#         songid, artistid = results
#     else:
#         songid, artistid = None, None


0 artist                                                Miami Horror
auth                                                     Logged In
firstName                                                     Kate
gender                                                           F
itemInSession                                                   88
lastName                                                   Harrell
length                                                    250.8273
level                                                         paid
location                                  Lansing-East Lansing, MI
method                                                         PUT
page                                                      NextSong
registration                                       1540472624796.0
sessionId                                                      293
song                                                     Sometimes
status                                                      

In [30]:
cur.execute(song_select,)

In [31]:
results = cur.fetchone()

In [32]:
print(results)

None


In [29]:
print(song_select)

SELECT songs.song_id, artists.artist_id FROM songs
                  JOIN artists ON songs.artist_id = artist.artist_id 
                  WHERE songs.song_title = %s AND songs.duration = %s 


#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [28]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.